In [ ]:
#Disini preprocessing
#Installasi untuk kebutuhan vnv
#gensim
#ntlk
#pyclustering

#jangan ubah jadi csv atau excel karena akan berupa koma-koma
#Kamis kita coba buat approach Bu Marketing



import os
import random
import re
import string

import nltk
import numpy as np
import pandas as pd

from gensim.models import Word2Vec, KeyedVectors
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import warnings
import numpy as np
np.warnings = warnings

import joblib


import os
import pandas as pd
path= os.path.join(os.getcwd(), 'Scraping_Dataset')
data = os.path.join(path,'com.kopikenangan.csv')
data = pd.read_csv(data).dropna(subset='content', ignore_index=True)

# Seed for reproducibility
SEED = 42
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)

# Download and define stopwords


stop_words = set(stopwords.words('english'))

def clean_text(text):
    """Pre-process text and generate tokens"""
    text = str(text).lower()  # Lowercase words
    text = re.sub(r"\[(.*?)\]", "", text)  # Remove [+XYZ chars] in content
    text = re.sub(r"\s+", " ", text)  # Remove multiple spaces in content
    text = re.sub(r"\w+…|…", "", text)  # Remove ellipsis (and last word)
    #text = re.sub() #melakukan trimp
    #text = re.sub() #menghilangkan kata yang hanya memeliki satu saja misal good dan lain-lain
    text = re.sub(r"(?<=\w)-(?=\w)", " ", text)  # Replace dash between words
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)  # Remove punctuation
    text = word_tokenize(text)
    text = [word for word in text if word not in stop_words]  # Remove stopwords
    return text
data['clean_comment']=data['content'].apply(clean_text)





In [20]:
data['content'].to_excel('googletranslate.xlsx')

In [ ]:
data_for_LDA=pd.read_excel('gt_kopikenangan.xlsx')
def clean_text(text):
    """Pre-process text and generate tokens"""
    text = str(text).lower()  # Lowercase words
    text = re.sub(r"\[(.*?)\]", "", text)  # Remove [+XYZ chars] in content
    text = re.sub(r"\s+", " ", text)  # Remove multiple spaces in content
    text = re.sub(r"\w+…|…", "", text)  # Remove ellipsis (and last word)
    #text = re.sub() #melakukan trimp
    #text = re.sub() #menghilangkan kata yang hanya memeliki satu saja misal good dan lain-lain
    text = re.sub(r"(?<=\w)-(?=\w)", " ", text)  # Replace dash between words
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)  # Remove punctuation
    text = word_tokenize(text)
    text = [word for word in text if word not in stop_words]  # Remove stopwords
    text = [word for word in text if len(word)>2]
    return text
data_for_LDA['translate']=data_for_LDA['content'].apply(clean_text)
data_for_LDA = data_for_LDA[data_for_LDA['translate'].apply(len) > 1]

In [38]:
data_for_LDA.head()

,Unnamed: 0,content,translate
1,1,The application makes the service easier. You...,"[application, makes, service, easier, order, f..."
2,2,"The most favorite hangout place, Kopken at Bl...","[favorite, hangout, place, kopken, blok, plaza]"
3,3,"Very delicious, service ok, discount ok, prom...","[delicious, service, discount, promo]"
4,4,The Taste and Price All Good,"[taste, price, good]"
5,5,nice coffee,"[nice, coffee]"


In [ ]:
#kita menggunkaan text Blob disebabkan
# polarity mereka 
# update mereka yang 3 bulan lalu
# adapun vader update 2 tahun lalu 

In [ ]:

#disini LDA sentiment analysis 

#disini lainya

0                                                   [good]
1        [aplikasinya, mempermudah, pelayanan, bisa, pe...
2        [tempat, nongkrong, paling, favoritkopken, yg,...
3           [enak, bangetpelayanan, okdiskon, okpromo, ok]
4                                [taste, dan, price, good]
                               ...                        
33570                                              [okeee]
33571                                           [oke, lah]
33572                                               [good]
33573    [kode, referal, di, input, dimana, ini, hmm, t...
33574                                     [senang, sekali]
Name: clean_comment, Length: 33575, dtype: object

In [ ]:
def get_vector(text):
    text = clean_text(text)
    word_vectors = [model[word] for word in text if word in model]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(300)  # Asumsi bahwa dimensi vektor adalah 300

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Advan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Advan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Advan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
# Apply get_vector to each comment
data['vector'] = data['comment'].apply(get_vector)

#Melakukan X means Clustering
from pyclustering.cluster.xmeans import xmeans, kmeans_plusplus_initializer
from pyclustering.cluster.center_initializer import random_center_initializer
from pyclustering.utils import timedcall
from pyclustering.samples.definitions import SIMPLE_SAMPLES
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Mengambil vektor untuk clustering
vectors = data['vector'].tolist()

# Inisialisasi center pertama kali (dengan K=2, untuk X-Means akan berkembang sesuai dengan data)
initial_centers = kmeans_plusplus_initializer(vectors, 2).initialize()

# Inisialisasi X-Means
xmeans_instance = xmeans(data=vectors, initial_centers=initial_centers, kmax=20)

# Proses clustering
xmeans_instance.process()

# Mendapatkan hasil clustering
clusters = xmeans_instance.get_clusters()
centers = xmeans_instance.get_centers()

# Menambahkan label cluster ke DataFrame
labels = np.zeros(len(data), dtype=int)
for cluster_index, cluster in enumerate(clusters):
    for data_index in cluster:
        labels[data_index] = cluster_index

data['cluster'] = labels


#mealkukan extraksi untuk kata-kata noun

def extract_nouns(text):
    tagged_words = nltk.pos_tag(text)
    nouns = [word for word, pos in tagged_words if pos in ('NN', 'NNS', 'NNP', 'NNPS')]  # POS tags untuk noun
    return nouns

data['nouns'] = data['clean_comment'].apply(extract_nouns)
#send to joblib
joblib.dump(data, 'noun_extrac.pkl')

In [ ]:
import os
LDAvis_data_filepath = os.path.join('ldavis_prepared_'+str(total_topics))
corpus = [dictionary.doc2bow(text) for text in doc_clean]



